<a href="https://colab.research.google.com/github/AntonioHenriqueAC/tcc/blob/master/google_colab/Barcode_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Baixando as bibliotecas necessárias

In [0]:
!pip3 install tensorflow-gpu==1.13.1

In [0]:
!pip3 install imageai --upgrade

# 2. Baixando a CNN pré-treinada

In [0]:
!wget https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5

# 3. Conectando o drive ao colab


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# 4. Carregando os arquivos do colab

In [0]:
!unzip "/content/drive/My Drive/Colab/barcode.zip" 

# 5. Treinando o modelo

In [0]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="barcode")
trainer.setTrainConfig(object_names_array=["barcode"], batch_size=4, num_experiments=20, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()

# 6. Avaliando os modelos gerados

In [0]:
metrics = trainer.evaluateModel(model_path="barcode/models", json_path="barcode/json/detection_config.json", iou_threshold=0.5, object_threshold=0.3, nms_threshold=0.5)

# 7. Testando o modelo treinado

In [0]:
#Setando as variáveis

modelPath = "/content/barcode/models/detection_model-ex-006--loss-0004.966.h5"
JsonPath = "barcode/json/detection_config.json"
InputImage = "/content/drive/My Drive/Colab/Imagens_Originais/img6.jpeg"

In [0]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image = InputImage, 
                                             output_image_path="barcode-detected.jpg")

print("----------------------------")
print("Número " , ": ", "Porcentagem ", "      :    ", "box_points")
print("")


for detection in detections:
    print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])

# 8. Treinando modelo e recortando a imagem

In [0]:
from imageai.Detection.Custom import CustomObjectDetection

detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(modelPath)
detector.setJsonPath(JsonPath)
detector.loadModel()
detections, extracted_objects_array = detector.detectObjectsFromImage(input_image = InputImage, 
                                                                      output_image_path="barcode-detected.jpg",
                                                                      extract_detected_objects=True)

print("----------------------------")
print("Número " , ": ", "Porcentagem ", "      :    ", "box_points")
print("")

for detection, object_path in zip(detections, extracted_objects_array):
    print(object_path)
    print(detection["name"], "      : ", detection["percentage_probability"], " : ", detection["box_points"])
    print("----------------------------")

In [0]:
#Numero de tag na img6: 63

len(detections)

In [0]:
import os

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas"))
file_count_corridas = len(files)
print("Corridas: ", file_count_corridas)

path, dirs, files = next(os.walk("/content/drive/My Drive/Colab/Corridas/Corrida_" + str(file_count_corridas) +"/images_tags"))
file_count_tag = len(files)
print("tags: ", file_count_tag)

newFolderCorrida = file_count_corridas + 1

!mkdir "/content/drive/My Drive/Colab/Corridas/Corrida_" + newFolder
